In [0]:
import os
import sys
import scipy.misc
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from PIL import Image
from nst_utils import *
import numpy as np
import tensorflow as tf
import pprint
%matplotlib inline

In [0]:
pp = pprint.PrettyPrinter(indent=4)
model = load_vgg_model("pretrained-model/imagenet-vgg-verydeep-19.mat")
pp.pprint(model)

Content cost

In [0]:
def compute_content_cost(a_C, a_G):
    """    
    Arguments:
    a_C -- tensor of dimension (1, n_H, n_W, n_C), hidden layer activations representing content of the image C 
    a_G -- tensor of dimension (1, n_H, n_W, n_C), hidden layer activations representing content of the image G
    
    """
    m, n_H, n_W, n_C = a_G.get_shape().as_list()
    
    a_C_unrolled = tf.reshape(a_C, shape=[m, n_H*n_W, n_C])
    a_G_unrolled = tf.reshape(a_G, [m, n_H*n_W, n_C])
    
    # content cost
    J_content = tf.reduce_sum(tf.square(tf.subtract(a_C_unrolled, a_G_unrolled)))/(4*n_H*n_W*n_C)
    
    return J_content

Layer style cost

In [0]:
def compute_layer_style_cost(a_S, a_G):
    """
    Arguments:
    a_S -- tensor of dimension (1, n_H, n_W, n_C), hidden layer activations representing style of the image S 
    a_G -- tensor of dimension (1, n_H, n_W, n_C), hidden layer activations representing style of the image G
    
    """

    m, n_H, n_W, n_C = a_G.get_shape().as_list()
    
    a_S = tf.reshape(tf.transpose(a_S, perm = [0,3,1,2]), shape=[m,n_C,n_H*n_W])
    a_G = tf.reshape(tf.transpose(a_G, perm = [0,3,1,2]), shape=[m,n_C,n_H*n_W])

    GS = tf.matmul(a_S, tf.transpose(a_S, perm = [0,2,1]))
    GG = tf.matmul(a_G, tf.transpose(a_G, perm = [0,2,1]))

    # layer style cost
    J_style_layer = tf.reduce_sum(tf.square(tf.subtract(GS, GG)))/((2*n_C*n_H*n_W)*(2*n_C*n_H*n_W))
    
    return J_style_layer

Overall style cost

In [0]:
# style contribution weightage for every layer

STYLE_LAYERS = [
    ('conv1_1', 0.2),
    ('conv2_1', 0.2),
    ('conv3_1', 0.2),
    ('conv4_1', 0.2),
    ('conv5_1', 0.2)]

In [0]:
def compute_style_cost(model, STYLE_LAYERS):
    
    J_style = 0

    for layer_name, coeff in STYLE_LAYERS:

        out = model[layer_name]
        a_S = sess.run(out)
        
        # a_G to be the hidden layer activation from same layer. Here, a_G references model['conv4_2'] 
        # and isn't evaluated yet. Later in the code, we'll assign the image G as the model input, so that
        # when we run the session, this will be the activations drawn from the appropriate layer, with G as input.
        a_G = out
        
        J_style_layer = compute_layer_style_cost(a_S, a_G)
        J_style += coeff * J_style_layer

    return J_style

Total cost

In [0]:
def total_cost(J_content, J_style, alpha = 10, beta = 40):
    
    J = alpha*J_content + beta*J_style
    
    return J

Optimization

(after reading the content and style image, and generating the noise image(input image) to start with)

In [0]:
# Assigning the input of the model to be the "content" image
sess.run(model['input'].assign(content_image))

out = model['conv4_2']
a_C = sess.run(out)
a_G = out

# content cost
J_content = compute_content_cost(a_C, a_G)

In [0]:
# Assigning the input of the model to be the "style" image 
sess.run(model['input'].assign(style_image))

J_style = compute_style_cost(model, STYLE_LAYERS)

In [0]:
J = 10*J_content + 40*J_style

In [0]:
# defining optimizer
optimizer = tf.train.AdamOptimizer(2.0)
train_step = optimizer.minimize(J)

In [0]:
def model_nn(sess, input_image, num_iterations = 200):

    sess.run(tf.global_variables_initializer())
    model["input"].assign(input_image)
    
    for i in range(num_iterations):
    
        sess.run(train_step)
        
        # Computing the generated image by running the session on the current model['input']
        generated_image = sess.run(model["input"])

        # Print every 20 iteration.
        if i%20 == 0:
            Jt, Jc, Js = sess.run([J, J_content, J_style])
            print("Iteration " + str(i) + " :")
            print("total cost = " + str(Jt))
            print("content cost = " + str(Jc))
            print("style cost = " + str(Js))
            
            save_image("output/" + str(i) + ".png", generated_image)
    
    # saving last generated image
    save_image('output/generated_image.jpg', generated_image)
    
    return generated_image